In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.applications.efficientnet import EfficientNetB2
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Flatten, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG
from sklearn.model_selection import train_test_split

import os
import random
import numpy as np
import matplotlib.pyplot as plt


In [2]:
mannequin_path = os.path.join('../','eventsData/training/mannequin')
# not_mannequin_path = os.path.join('../','not_mannequin_filter')

mannequin_img_path = [os.path.join(mannequin_path,x) for x in os.listdir(mannequin_path)]
# not_mannequin_img_path = [os.path.join(not_mannequin_path,x) for x in os.listdir(not_mannequin_path)]

print(mannequin_img_path[:5])
# print(not_mannequin_img_path[:5])

random.shuffle(mannequin_img_path)
random.shuffle(mannequin_img_path)
# random.shuffle(not_mannequin_img_path)
# random.shuffle(not_mannequin_img_path)

print(mannequin_img_path[:5])
# print(not_mannequin_img_path[:5])

['../eventsData/training/mannequin/446.png', '../eventsData/training/mannequin/1458.png', '../eventsData/training/mannequin/1245.png', '../eventsData/training/mannequin/748.png', '../eventsData/training/mannequin/1716.png']
['../eventsData/training/mannequin/874.png', '../eventsData/training/mannequin/1271.png', '../eventsData/training/mannequin/1757.png', '../eventsData/training/mannequin/860.png', '../eventsData/training/mannequin/1301.png']


In [3]:
train_mannequin_img, test_mannequin_img = train_test_split(mannequin_img_path,test_size = 0.20,random_state=42)

In [4]:
train_datagen = IDG(preprocessing_function=preprocess_input)
test_datagen = IDG()

train_path = '../eventsData/training'
shape = (150,150,3)

In [5]:
train_generator = train_datagen.flow_from_directory(train_path,target_size=(shape[0],shape[1]),batch_size=32,class_mode='binary')


Found 3887 images belonging to 2 classes.


In [6]:
efficientB2_model = EfficientNetB2(include_top=False,weights='imagenet',input_shape=shape)
efficientB2_model.trainable=False
x = GlobalAveragePooling2D()(efficientB2_model.output)
x = Dense(512,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1,activation='sigmoid')(x)

model = Model(efficientB2_model.input,x)
model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 150, 150, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 150, 150, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 151, 151, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

In [7]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])

In [8]:
model.fit(train_generator,epochs=20)

Epoch 1/20
122/122 [==============================] - 4s 32ms/step - loss: 0.2343 - accuracy: 0.9218
Epoch 2/20
122/122 [==============================] - 4s 32ms/step - loss: 0.0818 - accuracy: 0.9753
Epoch 3/20
122/122 [==============================] - 4s 32ms/step - loss: 0.0526 - accuracy: 0.9840
Epoch 4/20
122/122 [==============================] - 4s 31ms/step - loss: 0.0424 - accuracy: 0.9882
Epoch 5/20
122/122 [==============================] - 4s 32ms/step - loss: 0.0314 - accuracy: 0.9918
Epoch 6/20
122/122 [==============================] - 4s 31ms/step - loss: 0.0292 - accuracy: 0.9918
Epoch 7/20
122/122 [==============================] - 4s 31ms/step - loss: 0.0214 - accuracy: 0.9951
Epoch 8/20
122/122 [==============================] - 4s 32ms/step - loss: 0.0210 - accuracy: 0.9946
Epoch 9/20
122/122 [==============================] - 4s 31ms/step - loss: 0.0191 - accuracy: 0.9946
Epoch 10/20
122/122 [==============================] - 4s 32ms/step - loss: 0.0172 - accura